# 从头构建一个中文 RAG 系统

1. 采用 `gte-large-zh` 实现中文嵌入（Embedding）

2. 自定义 `TextEmbeddingFunction` 实现 lancedb 的自动嵌入

3. 采用 Volcengine 提供的 DeepSeek V3 实现 LLM 对话


## 读取数据


In [1]:
# 读取数据
with open("lease-zh.txt", "r", encoding="utf-8") as f:
    text_data = f.read()

## 数据分块（Chunk）


In [10]:
import re


def recursive_text_splitter_zh(text, max_chunk_length=300, overlap=50):
    """
    中文文本分块函数，保证语义完整性

    工作原理:
    1. 将半角标点符号转换为全角标点符号
    2. 定义中文标点符号和换行符作为分隔符
    3. 使用正则表达式按分隔符分割文本，同时保留分隔符
    4. 遍历分割后的文本:
        - 计算当前块的结束位置
        - 在标点符号处对齐块的边界，保证语义完整性
        - 考虑块之间的重叠，在重叠区域内查找合适的分割点
    5. 返回分块后的文本列表

    Args:
        text: 输入文本
        max_chunk_length: 每个块的最大字符数
        overlap: 块之间的重叠字符数
    """
    # 初始化结果
    result = []

    # 将半角标点符号转换为全角标点符号
    punctuation_map = {
        ",": "，",
        ".": "。",
        "!": "！",
        "?": "？",
        ";": "；",
        ":": "：",
        "(": "（",
        ")": "）",
        "[": "【",
        "]": "】",
    }
    for half, full in punctuation_map.items():
        text = text.replace(half, full)

    # 使用标点符号和换行符作为分隔符
    separators = ["\n", "。", "！", "？", "；", "：", "，", "、"]

    # 使用正则表达式分割文本，保留分隔符
    pattern = f"([{''.join(separators)}])"
    _splits = re.split(pattern, text)
    splits = [
        _splits[i] + (_splits[i + 1] if i + 1 < len(_splits) else "")
        for i in range(0, len(_splits), 2)
    ]

    current_pos = 0
    while current_pos < len(splits):
        # 计算当前块的结束位置
        current_chunk = ""
        current_length = 0
        end_pos = current_pos

        # 逐句添加,直到达到最大长度
        while (
            end_pos < len(splits)
            and current_length + len(splits[end_pos]) <= max_chunk_length
        ):
            current_chunk += splits[end_pos]
            current_length += len(splits[end_pos])
            end_pos += 1

        # 如果当前块为空(说明单个句子就超过了最大长度),则至少取一个句子
        if not current_chunk:
            current_chunk = splits[current_pos]
            end_pos = current_pos + 1

        # 添加当前块到结果中
        if current_chunk:
            result.append(current_chunk)

        # 计算下一个块的起始位置,考虑overlap
        if end_pos < len(splits):
            # 从当前块末尾向前找overlap个字符
            overlap_chars = 0
            overlap_pos = end_pos - 1
            while overlap_pos > current_pos and overlap_chars < overlap:
                overlap_chars += len(splits[overlap_pos])
                overlap_pos -= 1
            current_pos = overlap_pos + 1
        else:
            break

    return result


def verify_chunk_length(chunk, tokenizer):
    """
    验证chunk的token数是否超过限制
    """
    tokens = tokenizer.encode(chunk)
    return len(tokens) <= 512


# 使用示例
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("thenlper/gte-large-zh")

# 对文本进行分块
chunks = recursive_text_splitter_zh(text_data)

# 验证每个chunk
for i, chunk in enumerate(chunks):
    if not verify_chunk_length(chunk, tokenizer):
        print(f"警告：Chunk {i+1} 的token数超过限制")
    print(
        f"\nChunk {i+1}\n---------\n{chunk[:50]}...{chunk[-50:]}"
    )  # 只打印前50个字符和后50个字符


Chunk 1
---------

商业租赁协议

本《租赁协议》由以下双方于2013年12月1日签署生效：

出租方：Temple ...租赁并接受该物业；该租赁物业包括其上的所有附属建筑和改进部分。

第二条：租赁期限

第1节：租期


Chunk 2
---------
承租方亦同意租赁并接受该物业；该租赁物业包括其上的所有附属建筑和改进部分。

第二条：租赁期限

第...2月1日。

第三条：续期

本协议双方可选择续签，并就续签的条款和条件另行书面达成一致并签署文件。

Chunk 3
---------
“起始日期”指2013年12月1日。

第三条：续期

本协议双方可选择续签，并就续签的条款和条件另...。若租赁物业被真实出售，出租方有权将该保证金转交买方，并解除其归还义务。

第六条：税费

第1节：

Chunk 4
---------
则该金额将全额退还。若租赁物业被真实出售，出租方有权将该保证金转交买方，并解除其归还义务。

第六条...任何动产或不动产税提出异议或申请减税。若出租方同意，可配合签署必要文件。

第4节：评估费用的缴纳


Chunk 5
---------
承租方可自行承担费用对任何动产或不动产税提出异议或申请减税。若出租方同意，可配合签署必要文件。

第...单应包括出租方作为被保险人，保险证明应在租赁开始前提供，并在每次续保时更新。

第2节：出租方保险


Chunk 6
---------
保险单应包括出租方作为被保险人，保险证明应在租赁开始前提供，并在每次续保时更新。

第2节：出租方保...造成。

第九条：损害与毁损

若租赁物业遭受部分损毁，且仍适于营业用途，承租方应继续履行租约义务。

Chunk 7
---------
除非该损坏由承租方或其代表造成。

第九条：损害与毁损

若租赁物业遭受部分损毁，且仍适于营业用途，...方不得转让、分租或以其他方式处分本租约或其权益，亦不得允许第三方占用物业。

第十二条：进入权利



Chunk 8
---------
承租方不得转让、分租或以其他方式处分本租约或其权益，亦不得允许第三方占用物业。

第十二条：进入权利...送达方式。

第十五条：适用法律

本租约应受加利福尼亚州法律管辖与解释。

第十六条：完整协议



Chunk 9
-------

## 嵌入（Embedding）
